In [36]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-rqujap3w
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-rqujap3w
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-cp36-none-any.whl size=4308 sha256=588252be39da6791c558637aab4a5888f718e24bd08f37ead89bc2f91a83a126
  Stored in directory: /tmp/pip-ephem-wheel-cache-aog_9lij/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin
The nvcc_plugin extension is already loaded. To reload it, use:
  %reload_ext nvcc_plugin


**Q1**

In [ ]:
%%cu
#include<stdio.h>
__managed__ int sum=0;
__global__ void Array_sum(int *a,int *n)
{
    int tid=threadIdx.x;
    if(tid<*n)
      atomicAdd(&sum,a[tid]);
}

int main()
{
    int n=10,i;
    //printf("Enter N:");
    //scanf("%d",&n);
    int a[n];
    int *cuda_a,*cuda_n;
 
    for(i=0;i<n;i++)
    {
        a[i]=rand()%100;
        printf("%d ",a[i]);
    }
    printf("\n");

    cudaMalloc((void**)&cuda_a,n*sizeof(int));
    cudaMalloc((void**)&cuda_n,sizeof(int));
    
    cudaMemcpy(cuda_a,a,n*sizeof(int),cudaMemcpyHostToDevice);
    cudaMemcpy(cuda_n,&n,sizeof(int),cudaMemcpyHostToDevice);
    Array_sum <<<1,n>>>(cuda_a,cuda_n);
    printf("Sum:%d\n",sum);
    cudaFree(cuda_a);
    cudaFree(cuda_n);
    return 0;
}

83 86 77 15 93 35 86 92 49 21 
Sum:637



**Q2**

In [ ]:
%%cu
#include<stdio.h>

__global__ void Array_add(int *a,int *b,int *c,int *n)
{
    unsigned short tid=threadIdx.x;
    if(tid<*n)
      c[tid]=a[tid]+b[tid];
}

int main()
{
    int n=5,i;
    //printf("Enter N:");
    //scanf("%d",&n);
    int a[n],b[n],c[n];
    int *cuda_a,*cuda_b,*cuda_c,*cuda_n;
 
    for(i=0;i<n;i++)
      a[i]=rand()%100;
    for(i=0;i<n;i++)
      b[i]=rand()%100;

    cudaMalloc((void**)&cuda_a,n*sizeof(int));
    cudaMalloc((void**)&cuda_b,n*sizeof(int));
    cudaMalloc((void**)&cuda_c,n*sizeof(int));
    cudaMalloc((void**)&cuda_n,sizeof(int));
    
    cudaMemcpy(cuda_a,a,n*sizeof(int),cudaMemcpyHostToDevice);
    cudaMemcpy(cuda_b,b,n*sizeof(int),cudaMemcpyHostToDevice);
    cudaMemcpy(cuda_n,&n,sizeof(int),cudaMemcpyHostToDevice);
    Array_add <<<1,n>>>(cuda_a,cuda_b,cuda_c,cuda_n);
    cudaMemcpy(c,cuda_c,n*sizeof(int),cudaMemcpyDeviceToHost);
    for(i=0;i<n;i++)
      printf("%d + %d = %d\n",a[i],b[i],c[i]);
    cudaFree(cuda_a);
    cudaFree(cuda_b);
    cudaFree(cuda_c);
    cudaFree(cuda_n);
    return 0;
}

83 + 35 = 118
86 + 86 = 172
77 + 92 = 169
15 + 49 = 64
93 + 21 = 114



**Q3**

In [52]:
%%cu
#include<stdio.h>

__global__ void Matrix_mult(int *a,int *b,int *c,int *m,int *n,int *p)
{
    int col=blockIdx.y*blockDim.y+threadIdx.y;
    int row=blockIdx.x*blockDim.x+threadIdx.x;
    int temp=0,i;

    if(row<*m&&col<*p)
    for(i=0;i<*n;i++)
      temp+=a[row*(*n)+i]*b[i*(*p)+col];
    c[row*(*p)+col]=temp;
}

int main()
{
    int m=3,n=2,p=2,i,j;
    //printf("Enter N:");
    //scanf("%d",&n);
    int a[m*n],b[n*p],c[m*n];
    int *cuda_a,*cuda_b,*cuda_c,*cuda_m,*cuda_n,*cuda_p;
    printf("Matrix A:\n");
    for(i=0;i<m;i++)
    {
        for(j=0;j<n;j++)
        {
            a[i*n+j]=rand()%100;
            printf("%d ",a[i*n+j]);       
        }
        printf("\n");
    }
    printf("\nMatrix B:\n");
    for(i=0;i<n;i++)
    {
        for(j=0;j<p;j++)
        {
            b[i*p+j]=rand()%100;
            printf("%d ",b[i*p+j]);       
        }
        printf("\n");
    }
    printf("\n");

    cudaMalloc((void**)&cuda_a,m*n*sizeof(int));
    cudaMalloc((void**)&cuda_b,n*p*sizeof(int));
    cudaMalloc((void**)&cuda_c,m*p*sizeof(int));
    cudaMalloc((void**)&cuda_m,sizeof(int));
    cudaMalloc((void**)&cuda_n,sizeof(int));
    cudaMalloc((void**)&cuda_p,sizeof(int));

    cudaMemcpy(cuda_a,a,m*n*sizeof(int),cudaMemcpyHostToDevice);
    cudaMemcpy(cuda_b,b,n*p*sizeof(int),cudaMemcpyHostToDevice);
    cudaMemcpy(cuda_m,&m,sizeof(int),cudaMemcpyHostToDevice);
    cudaMemcpy(cuda_n,&n,sizeof(int),cudaMemcpyHostToDevice);
    cudaMemcpy(cuda_p,&p,sizeof(int),cudaMemcpyHostToDevice);
 
    dim3 threadsPerBlock(m,p);
    dim3 blocksPerGrid(1,1);
        

    Matrix_mult<<<blocksPerGrid,threadsPerBlock>>> (cuda_a,cuda_b,cuda_c,cuda_m,cuda_n,cuda_p);   

    cudaMemcpy(c,cuda_c,m*p*sizeof(int),cudaMemcpyDeviceToHost);
    printf("Result Matrix:\n");
    for(i=0;i<m;i++)
    {
        for(j=0;j<p;j++)
        {
            printf("%d ",c[i*p+j]);       
        }
        printf("\n");
    }
    cudaFree(cuda_a);
    cudaFree(cuda_b);
    cudaFree(cuda_c);
    cudaFree(cuda_m);
    cudaFree(cuda_n);
    cudaFree(cuda_p);
    return 0;
}

Matrix A:
83 86 
77 15 
93 35 

Matrix B:
86 92 
49 21 

Result Matrix:
11352 9442 
7357 7399 
9713 9291 



**Q5**

In [ ]:
%%cu
#include<stdio.h>
__managed__ int n,ne;
__global__ void BFS(int *v,int *e,bool *f,bool *x,int *c)
{
    int tid=threadIdx.x;
    if(tid<n)
    {
        if(f[tid]==true)
        {
            f[tid]=false;
            x[tid]=true;
            int start=v[tid];
            int end=2*ne;
            if(tid!=n-1)
              end=v[tid+1];
            for(int i=start;i<end;i++)
            {
                int temp=e[i];
                if(x[temp]==false)
                {
                    c[temp]=c[tid]+1;
                    f[temp]=true;
                }
            }
        }
    }
}

int main()
{
    int s=2;// source vertex
    int i,j,k=0;
    n=5;
    ne=0;
  //generate random graph
  int gr[n][n];
  for(i=0;i<n;i++)
  {
      for(j=0;j<n;j++)
      {
          if(i==j)
          {
              gr[i][j]=0;
          }
          else if(i>j)
          {
              gr[i][j]=gr[j][i];
          }
          else
          {
              gr[i][j]=rand()%2;
          }
          if(gr[i][j]!=0)
            ne++;
          printf("%d ",gr[i][j]);
      }
      printf("\n");
  }
    ne/=2;
    //printf("Enter N:");
    //scanf("%d",&n);
    //int *v=malloc((int*)&v,n*sizeof(int));
    //int *e=malloc((int*)&e,2*ne*sizeof(int));
    int v[n],e[2*ne];
    for(i=0;i<n;i++)
    {
        v[i]=k;
        for(j=0;j<n;j++)
        {
            if(gr[i][j]!=0)
            {
                e[k]=j;
                k++;
            }
        }
    }
    //bool *f=malloc((bool*)&f,n*sizeof(bool));
    //bool *x=malloc((bool*)&x,n*sizeof(bool));
    //int *c=malloc((int*)&c,n*sizeof(int));
    bool f[n],x[n];
    int c[n];
    for(i=0;i<n;i++)
    {
        f[i]=x[i]=false;
        c[i]=INT_MAX;
    }
    c[s]=0;
    f[s]=true;
    int *cuda_v,*cuda_e,*cuda_c;
    bool *cuda_f,*cuda_x;
    cudaMalloc((void**)&cuda_v,n*sizeof(int));
    cudaMalloc((void**)&cuda_e,2*ne*sizeof(int));
    cudaMalloc((void**)&cuda_f,n*sizeof(bool));
    cudaMalloc((void**)&cuda_x,n*sizeof(bool));
    cudaMalloc((void**)&cuda_c,n*sizeof(int));
    

    cudaMemcpy(cuda_v,v,n*sizeof(int),cudaMemcpyHostToDevice);
    cudaMemcpy(cuda_e,e,2*ne*sizeof(int),cudaMemcpyHostToDevice);
    cudaMemcpy(cuda_f,f,n*sizeof(bool),cudaMemcpyHostToDevice);
    cudaMemcpy(cuda_x,x,n*sizeof(bool),cudaMemcpyHostToDevice);
    cudaMemcpy(cuda_c,c,n*sizeof(int),cudaMemcpyHostToDevice);
    while(1)
    {
        int flag=0;
        BFS <<<1,n>>>(cuda_v,cuda_e,cuda_f,cuda_x,cuda_c);
        cudaDeviceSynchronize();
        cudaMemcpy(f,cuda_f,n*sizeof(bool),cudaMemcpyDeviceToHost);
        for(i=0;i<n;i++)
        {
            if(f[i]==true)
            {
                flag=1;
                break;
            }
        }
        if(flag==0)
          break;
    }
    cudaMemcpy(c,cuda_c,n*sizeof(int),cudaMemcpyDeviceToHost);
    printf("\nDistance from vertex %d:\n",s);
    for(i=0;i<n;i++)
        printf("%d ",c[i]);
    cudaFree(cuda_v);
    cudaFree(cuda_e);
    cudaFree(cuda_f);
    cudaFree(cuda_x);
    cudaFree(cuda_c);
    return 0;
}

0 1 0 1 1 
1 0 1 1 0 
0 1 0 0 1 
1 1 0 0 1 
1 0 1 1 0 

Distance from vertex 2:
2 1 0 2 1 


**Q6**

In [ ]:
%%cu
#include<stdio.h>
__managed__ int n,ne;
__global__ void SSSP(int *v,int *e,int *w,bool *m,int *u,int *c)
{
    int tid=threadIdx.x;
    if(tid<n)
    {
        if(m[tid]==true)
        {
            m[tid]=false;
            int start=v[tid];
            int end=2*ne;
            if(tid!=n-1)
              end=v[tid+1];
            for(int i=start;i<end;i++)
            {
                int temp=e[i];
                if(u[temp]>c[tid]+w[temp])
                {
                    u[temp]=c[tid]+w[temp];
                }
            }
        }
    }
}

__global__ void SSSP_Kernel(int *v,int *e,int *w,bool *m,int *u,int *c)
{
    int tid=threadIdx.x;
    if(tid<n)
    {
        if(c[tid]>u[tid])
        {
            c[tid]=u[tid];
            m[tid]=true;
        }
        u[tid]=c[tid];
    }
}

int main()
{
    int s=2;// source vertex
    int i,j,k=0;
    n=5;
    ne=0;
  //generate random graph
  int gr[n][n];
  for(i=0;i<n;i++)
  {
      for(j=0;j<n;j++)
      {
          if(i==j)
          {
              gr[i][j]=0;
          }
          else if(i>j)
          {
              gr[i][j]=gr[j][i];
          }
          else
          {
              gr[i][j]=rand()%100;
          }
          if(gr[i][j]!=0)
            ne++;
          printf("%d ",gr[i][j]);
      }
      printf("\n");
  }
    ne/=2;
    //printf("Enter N:");
    //scanf("%d",&n);
    //int *v=malloc((int*)&v,n*sizeof(int));
    //int *e=malloc((int*)&e,2*ne*sizeof(int));
    int v[n],e[2*ne],w[2*ne];
    for(i=0;i<n;i++)
    {
        v[i]=k;
        for(j=0;j<n;j++)
        {
            if(gr[i][j]!=0)
            {
                e[k]=j;
                w[k]=gr[i][j];
                k++;
            }
        }
    }
    //bool *f=malloc((bool*)&f,n*sizeof(bool));
    //bool *x=malloc((bool*)&x,n*sizeof(bool));
    //int *c=malloc((int*)&c,n*sizeof(int));
    bool m[n];
    int c[n],u[n];
    for(i=0;i<n;i++)
    {
        m[i]=false;
        c[i]=u[i]=INT_MAX;
    }
    c[s]=0;
    u[s]=0;
    m[s]=true;
    int *cuda_v,*cuda_e,*cuda_c,*cuda_u,*cuda_w;
    bool *cuda_m;
    cudaMalloc((void**)&cuda_v,n*sizeof(int));
    cudaMalloc((void**)&cuda_e,2*ne*sizeof(int));
    cudaMalloc((void**)&cuda_w,2*ne*sizeof(int));
    cudaMalloc((void**)&cuda_m,n*sizeof(bool));
    cudaMalloc((void**)&cuda_u,n*sizeof(int));
    cudaMalloc((void**)&cuda_c,n*sizeof(int));
    

    cudaMemcpy(cuda_v,v,n*sizeof(int),cudaMemcpyHostToDevice);
    cudaMemcpy(cuda_e,e,2*ne*sizeof(int),cudaMemcpyHostToDevice);
    cudaMemcpy(cuda_w,w,2*ne*sizeof(int),cudaMemcpyHostToDevice);
    cudaMemcpy(cuda_m,m,n*sizeof(bool),cudaMemcpyHostToDevice);
    cudaMemcpy(cuda_u,u,n*sizeof(int),cudaMemcpyHostToDevice);
    cudaMemcpy(cuda_c,c,n*sizeof(int),cudaMemcpyHostToDevice);
    while(1)
    {
        int flag=0;
        SSSP <<<1,n>>>(cuda_v,cuda_e,cuda_w,cuda_m,cuda_u,cuda_c);
        cudaDeviceSynchronize();
        SSSP_Kernel <<<1,n>>>(cuda_v,cuda_e,cuda_w,cuda_m,cuda_u,cuda_c);
        cudaDeviceSynchronize();
        cudaMemcpy(m,cuda_m,n*sizeof(bool),cudaMemcpyDeviceToHost);
        for(i=0;i<n;i++)
        {
            if(m[i]==true)
            {
                flag=1;
                break;
            }
        }
        if(flag==0)
          break;
    }
    cudaMemcpy(c,cuda_c,n*sizeof(int),cudaMemcpyDeviceToHost);
    printf("\nDistance from vertex %d:\n",s);
    for(i=0;i<n;i++)
        printf("%d ",c[i]);
    cudaFree(cuda_v);
    cudaFree(cuda_e);
    cudaFree(cuda_w);
    cudaFree(cuda_m);
    cudaFree(cuda_u);
    cudaFree(cuda_c);
    return 0;
}

0 83 86 77 15 
83 0 93 35 86 
86 93 0 92 49 
77 35 92 0 21 
15 86 49 21 0 

Distance from vertex 2:
83 86 0 15 83 


**Q7**

In [ ]:
%%cu
#include<thrust/host_vector.h>
#include<thrust/device_vector.h>
#include<thrust/generate.h>
#include<thrust/sort.h>
#include<thrust/copy.h>
#include<cstdlib>

int main()
{
    thrust::host_vector<int> a(10);
    thrust::generate(a.begin(),a.end(),rand);
    thrust::device_vector<int> device_a=a;
    thrust::sort(device_a.begin(),device_a.end());
    thrust::copy(device_a.begin(),device_a.end(),a.begin());
    for(auto x:a)
      printf("%d ",x);
    return 0;
}

424238335 596516649 719885386 846930886 1189641421 1649760492 1681692777 1714636915 1804289383 1957747793 
